In [2]:
# STEP 1: Install required libraries
!pip install -q simpletransformers
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00
 

In [3]:
# STEP 2: Import libraries
import pandas as pd
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split

In [4]:
!pip install -q scikit-learn
!pip install -q tensorflow

In [5]:
# STEP 2: Imports
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [6]:
# STEP 3: Mount Google Drive (if data is stored there)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# STEP 4: Load and prepare data
# Modify this path to where your OCR text files are stored
train_path = "/content/drive/MyDrive/meme_classifications/ocr_text/train/STask_C_train.csv"
eval_path = "/content/drive/MyDrive/meme_classifications/ocr_text/eval/STask-C(index,text)val.csv"

In [8]:
train_df = pd.read_csv(train_path)
eval_df = pd.read_csv(eval_path)

In [9]:
# Confirm the structure
print(train_df.head())
print(eval_df.head())

       index                                               text  label
0  30003.png  gender envy from guys gender envy from cis gir...      0
1  30004.png  will things in the world ever get better for t...      1
2  30005.png  mgflip com gender identity gender expression c...      2
3  30007.png  ben shapiro character ai remember everything c...      1
4  30012.png  follow many cancelling disney to protect their...      2
       index                                               text
0  40010.png    when you make a joke but it's 2019 vegan ch e r
1  40030.png  trans people i'm replacing my hormones and fig...
2  40052.png  apollo for president 2024 scan me trans rights...
3  40066.png  when france is saying that they support gay ri...
4  40086.png  bbc news bbc news bbc news 1 hr it's a good da...


In [10]:
# STEP 5: Preprocessing for ALBERT
train_df = train_df.rename(columns={"text": "text", "label": "labels"})
train_data, val_data = train_test_split(train_df[["text", "labels"]], test_size=0.1, random_state=42)

In [11]:
# STEP 6: ALBERT Model
model_args = ClassificationArgs()
model_args.num_train_epochs = 4
model_args.train_batch_size = 16
model_args.eval_batch_size = 32
model_args.max_seq_length = 128
model_args.output_dir = "outputs/subtaskC"
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.use_early_stopping = True
model_args.early_stopping_consider_epochs = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_patience = 2
model_args.logging_steps = 50

num_labels = train_df.labels.nunique()

albert_model = ClassificationModel(
    "albert",
    "albert-base-v2",
    num_labels=num_labels,
    args=model_args,
    use_cuda=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [12]:
# Train ALBERT
albert_model.train_model(train_data, eval_df=val_data)

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


...

Running Epoch 4 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(912,
 defaultdict(list,
             {'global_step': [228, 456, 684, 912],
              'train_loss': [0.999098539352417,
               1.1248873472213745,
               1.042311429977417,
               0.9984788298606873],
              'mcc': [0.0,
               np.float64(0.11110253117026066),
               np.float64(0.11192843435164844),
               np.float64(0.23634595334576652)],
              'eval_loss': [1.1068788766860962,
               1.082873055568108,
               1.0439370687191303,
               1.0087136351145232]}))

In [18]:
from transformers import AlbertTokenizer
import torch
from scipy.special import softmax
from tqdm import tqdm


In [19]:
# STEP 8: Predict ALBERT on eval set — Get softmax probabilities from logits
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

inputs = tokenizer(
    eval_df["text"].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = albert_model.model.to(device)
model.eval()

input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Convert to probabilities
albert_probs = softmax(logits.cpu().numpy(), axis=1)


In [20]:
# STEP 7: ANN with TF-IDF features
tfidf = TfidfVectorizer(max_features=3000)
X_tfidf = tfidf.fit_transform(train_df["text"]).toarray()
X_eval_tfidf = tfidf.transform(eval_df["text"]).toarray()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train_df["labels"])
y_cat = to_categorical(y_encoded)

In [21]:
# Build ANN model
ann_model = Sequential()
ann_model.add(Dense(256, activation='relu', input_shape=(X_tfidf.shape[1],)))
ann_model.add(Dropout(0.3))
ann_model.add(Dense(128, activation='relu'))
ann_model.add(Dropout(0.3))
ann_model.add(Dense(num_labels, activation='softmax'))

ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
# Train ANN
ann_model.fit(X_tfidf, y_cat, epochs=10, batch_size=32, validation_split=0.1, verbose=1)


Epoch 1/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.3956 - loss: 1.0798 - val_accuracy: 0.5086 - val_loss: 0.9881
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6787 - loss: 0.8051 - val_accuracy: 0.4988 - val_loss: 1.0170
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8369 - loss: 0.4981 - val_accuracy: 0.4790 - val_loss: 1.2081
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9215 - loss: 0.2695 - val_accuracy: 0.4420 - val_loss: 1.5239
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9659 - loss: 0.1380 - val_accuracy: 0.4741 - val_loss: 1.8376
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9828 - loss: 0.0806 - val_accuracy: 0.4593 - val_loss: 2.0880
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9870 - loss: 0.0531 - val_accuracy: 0.4667 - val_loss: 2.1798
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9913 - loss: 0.0394 - val_accuracy: 0

In [23]:
# Predict ANN on eval
ann_probs = ann_model.predict(X_eval_tfidf)
ann_preds = np.argmax(ann_probs, axis=1)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [24]:
# STEP 8: Weighted Ensemble (ALBERT + ANN)
# Normalize predictions
albert_probs_norm = albert_probs / albert_probs.sum(axis=1, keepdims=True)
ann_probs_norm = ann_probs / ann_probs.sum(axis=1, keepdims=True)

In [25]:
# Weighted average (adjust weights if needed)
ensemble_probs = 0.7 * albert_probs_norm + 0.3 * ann_probs_norm
ensemble_preds = np.argmax(ensemble_probs, axis=1)

In [26]:
# Decode predictions if needed
final_labels = ensemble_preds

In [27]:
# STEP 9: Save submission
submission_df = pd.DataFrame({
    "index": eval_df["index"],
    "label": final_labels
})

In [28]:
submission_path = "/content/drive/MyDrive/meme_classifications/submissions/subtaskC_submission.csv"
submission_df.to_csv(submission_path, index=False)

print("✅ Submission saved to:", submission_path)


✅ Submission saved to: /content/drive/MyDrive/meme_classifications/submissions/subtaskC_submission.csv
